# 实验

In [1]:
import pickle
import warnings
warnings.filterwarnings('ignore')

# 直读取最优特征数据集
with open('../data/processed/experiment_data_refined.pkl', 'rb') as f:
    experiment_data_refined = pickle.load(f)

print("The filtered feature data has been successfully loaded from experiment_data_refined.pkl.")

The filtered feature data has been successfully loaded from experiment_data_refined.pkl.


## 模型实验（Baseline）

### ARIMAX（带有外生变量的 ARIMA）

In [2]:
import numpy as np
import statsmodels.api as sm
import warnings

warnings.filterwarnings("ignore")

# 初始化全局存储结构
if 'all_results' not in globals():
    all_results = []
if 'all_errors' not in globals():
    all_errors = []

model_name = "ARIMAX"
current_results = {}

# 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
all_results = [res for res in all_results if res['model'] != model_name]
all_errors = [err for err in all_errors if err['model'] != model_name]

for w, data in experiment_data_refined.items():
    # 1. 数据展平处理 (N, L, D) -> (N, L*D) 适配传统计量模型
    N_tr, L, D = data['X_train'].shape
    X_train_flat = data['X_train'].reshape(N_tr, L * D)
    N_te = data['X_test'].shape[0]
    X_test_flat = data['X_test'].reshape(N_te, L * D)
    
    y_train = data['y_train'].flatten()
    y_true = data['y_test'].flatten()
    
    # 2. 模型构建与拟合
    model = sm.tsa.ARIMA(endog=y_train, exog=X_train_flat, order=(1, 0, 0))
    res = model.fit()
    
    # 3. 模型预测 (修正了.values的问题，直接转np.array并展平)
    y_pred = np.array(res.forecast(steps=len(y_true), exog=X_test_flat)).flatten()
    
    # 4. 基准预测 (零收益)
    y_pred_baseline = np.zeros_like(y_true)
    
    # 5. 指标计算
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae = np.mean(np.abs(y_true - y_pred))
    r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
    da = np.mean(np.sign(y_true) == np.sign(y_pred)) * 100
    
    # 6. 误差序列计算
    e_t = y_true - y_pred
    e0_t = y_true - y_pred_baseline
    
    # 7. 组装结果字典
    results_dict = {
        "model": model_name,
        "window": w,
        "RMSE": rmse,
        "MAE": mae,
        "R2_OS": r2_os,
        "DA": da
    }
    
    error_dict = {
        "model": model_name,
        "window": w,
        "errors": e_t,
        "benchmark_errors": e0_t
    }
    
    # 8. 存储结果
    all_results.append(results_dict)
    all_errors.append(error_dict)
    current_results[w] = results_dict

# ---------------- 输出格式模拟 ----------------
print("-" * 110)
# 表头第一行：窗口
header_win = f"{'Model':<15}"
# 表头第二行：指标
header_met = f"{'':<15}"
# 数据行
row_data = f"{model_name:<15}"

for w in experiment_data_refined.keys():
    header_win += f"{f'Window = {w}':^30}"
    header_met += f"{'RMSE':>7} {'MAE':>7} {'R2':>7} {'DA (%)':>7}  "
    
    res = current_results[w]
    row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

print(header_win)
print(header_met)
print("-" * 110)
print(row_data)
print("-" * 110)

--------------------------------------------------------------------------------------------------------------
Model                    Window = 5                   Window = 21                   Window = 63          
                  RMSE     MAE      R2  DA (%)     RMSE     MAE      R2  DA (%)     RMSE     MAE      R2  DA (%)  
--------------------------------------------------------------------------------------------------------------
ARIMAX          0.0113  0.0082 -0.0598   50.64   0.0117  0.0087 -0.1263   50.55   0.0130  0.0101 -0.4006   47.25  
--------------------------------------------------------------------------------------------------------------


### XGBoost

In [3]:
import numpy as np
import xgboost as xgb
import warnings

warnings.filterwarnings("ignore")

# 初始化全局存储结构
if 'all_results' not in globals():
    all_results = []
if 'all_errors' not in globals():
    all_errors = []

model_name = "XGBoost"
current_results = {}

# 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
all_results = [res for res in all_results if res['model'] != model_name]
all_errors = [err for err in all_errors if err['model'] != model_name]

for w, data in experiment_data_refined.items():
    # 1. 数据展平处理 (N, L, D) -> (N, L*D) 适配机器学习模型
    N_tr, L, D = data['X_train'].shape
    X_train_flat = data['X_train'].reshape(N_tr, L * D)
    N_te = data['X_test'].shape[0]
    X_test_flat = data['X_test'].reshape(N_te, L * D)
    
    y_train = data['y_train'].flatten()
    y_true = data['y_test'].flatten()
    
    # 2. 模型构建与拟合
    model = xgb.XGBRegressor(n_estimators=100, max_depth=6, random_state=42, n_jobs=-1)
    model.fit(X_train_flat, y_train)
    
    # 3. 模型预测
    y_pred = model.predict(X_test_flat).flatten()
    
    # 4. 基准预测 (零收益)
    y_pred_baseline = np.zeros_like(y_true)
    
    # 5. 指标计算
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae = np.mean(np.abs(y_true - y_pred))
    r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
    da = np.mean(np.sign(y_true) == np.sign(y_pred)) * 100
    
    # 6. 误差序列计算
    e_t = y_true - y_pred
    e0_t = y_true - y_pred_baseline
    
    # 7. 组装结果字典
    results_dict = {
        "model": model_name,
        "window": w,
        "RMSE": rmse,
        "MAE": mae,
        "R2_OS": r2_os,
        "DA": da
    }
    
    error_dict = {
        "model": model_name,
        "window": w,
        "errors": e_t,
        "benchmark_errors": e0_t
    }
    
    # 8. 存储结果
    all_results.append(results_dict)
    all_errors.append(error_dict)
    current_results[w] = results_dict

# ---------------- 输出格式模拟 ----------------
print("-" * 110)
# 表头第一行：窗口
header_win = f"{'Model':<15}"
# 表头第二行：指标
header_met = f"{'':<15}"
# 数据行
row_data = f"{model_name:<15}"

for w in experiment_data_refined.keys():
    header_win += f"{f'Window = {w}':^30}"
    header_met += f"{'RMSE':>7} {'MAE':>7} {'R2':>7} {'DA (%)':>7}  "
    
    res = current_results[w]
    row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

print(header_win)
print(header_met)
print("-" * 110)
print(row_data)
print("-" * 110)

--------------------------------------------------------------------------------------------------------------
Model                    Window = 5                   Window = 21                   Window = 63          
                  RMSE     MAE      R2  DA (%)     RMSE     MAE      R2  DA (%)     RMSE     MAE      R2  DA (%)  
--------------------------------------------------------------------------------------------------------------
XGBoost         0.0123  0.0092 -0.2751   50.43   0.0123  0.0094 -0.2599   50.76   0.0123  0.0092 -0.2376   52.75  
--------------------------------------------------------------------------------------------------------------


### LSTM

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import warnings

warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')
tf.random.set_seed(42)

# 初始化全局存储结构
if 'all_results' not in globals():
    all_results = []
if 'all_errors' not in globals():
    all_errors = []

model_name = "LSTM"
current_results = {}

# 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
all_results = [res for res in all_results if res['model'] != model_name]
all_errors = [err for err in all_errors if err['model'] != model_name]

for w, data in experiment_data_refined.items():
    # 1. 提取3D特征张量 (N, L, D)，深度学习天然支持该形状，无需展平
    X_train_3d = data['X_train']
    X_test_3d = data['X_test']
    
    y_train = data['y_train'].flatten()
    y_true = data['y_test'].flatten()
    
    N_tr, L, D = X_train_3d.shape
    
    # 2. 模型构建与拟合
    model = Sequential([
        LSTM(32, input_shape=(L, D)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    
    # 为了演示极简且可运行，设置epochs=50。设置shuffle=False保留时序特征
    model.fit(X_train_3d, y_train, epochs=50, batch_size=32, verbose=0, shuffle=False)
    
    # 3. 模型预测
    y_pred = model.predict(X_test_3d, verbose=0).flatten()
    
    # 4. 基准预测 (零收益)
    y_pred_baseline = np.zeros_like(y_true)
    
    # 5. 指标计算
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae = np.mean(np.abs(y_true - y_pred))
    r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
    da = np.mean(np.sign(y_true) == np.sign(y_pred)) * 100
    
    # 6. 误差序列计算
    e_t = y_true - y_pred
    e0_t = y_true - y_pred_baseline
    
    # 7. 组装结果字典
    results_dict = {
        "model": model_name,
        "window": w,
        "RMSE": rmse,
        "MAE": mae,
        "R2_OS": r2_os,
        "DA": da
    }
    
    error_dict = {
        "model": model_name,
        "window": w,
        "errors": e_t,
        "benchmark_errors": e0_t
    }
    
    # 8. 存储结果
    all_results.append(results_dict)
    all_errors.append(error_dict)
    current_results[w] = results_dict

# ---------------- 输出格式模拟 ----------------
print("-" * 110)
# 表头第一行：窗口
header_win = f"{'Model':<15}"
# 表头第二行：指标
header_met = f"{'':<15}"
# 数据行
row_data = f"{model_name:<15}"

for w in experiment_data_refined.keys():
    header_win += f"{f'Window = {w}':^30}"
    header_met += f"{'RMSE':>7} {'MAE':>7} {'R2_OS':>7} {'DA (%)':>7}  "
    
    res = current_results[w]
    row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

print(header_win)
print(header_met)
print("-" * 110)
print(row_data)
print("-" * 110)

--------------------------------------------------------------------------------------------------------------
Model                    Window = 5                   Window = 21                   Window = 63          
                  RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)  
--------------------------------------------------------------------------------------------------------------
LSTM            0.0155  0.0108 -1.0170   51.07   0.0163  0.0112 -1.2084   50.44   0.0223  0.0159 -3.0779   48.28  
--------------------------------------------------------------------------------------------------------------


### GRU

In [35]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
import warnings

warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')
tf.random.set_seed(42)

# 初始化全局存储结构
if 'all_results' not in globals():
    all_results = []
if 'all_errors' not in globals():
    all_errors = []

model_name = "GRU"
current_results = {}

# 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
all_results = [res for res in all_results if res['model'] != model_name]
all_errors = [err for err in all_errors if err['model'] != model_name]

for w, data in experiment_data_refined.items():
    # 1. 提取3D特征张量 (N, L, D)
    X_train_3d = data['X_train']
    X_test_3d = data['X_test']
    
    y_train = data['y_train'].flatten()
    y_true = data['y_test'].flatten()
    
    N_tr, L, D = X_train_3d.shape
    
    # 2. 模型构建与拟合
    model = Sequential([
        GRU(32, input_shape=(L, D)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    
    # 保持与LSTM一致的训练参数配置
    model.fit(X_train_3d, y_train, epochs=50, batch_size=32, verbose=0, shuffle=False)
    
    # 3. 模型预测
    y_pred = model.predict(X_test_3d, verbose=0).flatten()
    
    # 4. 基准预测 (零收益)
    y_pred_baseline = np.zeros_like(y_true)
    
    # 5. 指标计算
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae = np.mean(np.abs(y_true - y_pred))
    r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
    da = np.mean(np.sign(y_true) == np.sign(y_pred)) * 100
    
    # 6. 误差序列计算
    e_t = y_true - y_pred
    e0_t = y_true - y_pred_baseline
    
    # 7. 组装结果字典
    results_dict = {
        "model": model_name,
        "window": w,
        "RMSE": rmse,
        "MAE": mae,
        "R2_OS": r2_os,
        "DA": da
    }
    
    error_dict = {
        "model": model_name,
        "window": w,
        "errors": e_t,
        "benchmark_errors": e0_t
    }
    
    # 8. 存储结果
    all_results.append(results_dict)
    all_errors.append(error_dict)
    current_results[w] = results_dict

# ---------------- 输出格式模拟 ----------------
print("-" * 110)
# 表头第一行：窗口
header_win = f"{'Model':<15}"
# 表头第二行：指标
header_met = f"{'':<15}"
# 数据行
row_data = f"{model_name:<15}"

for w in experiment_data_refined.keys():
    header_win += f"{f'Window = {w}':^30}"
    header_met += f"{'RMSE':>7} {'MAE':>7} {'R2_OS':>7} {'DA (%)':>7}  "
    
    res = current_results[w]
    row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

print(header_win)
print(header_met)
print("-" * 110)
print(row_data)
print("-" * 110)

--------------------------------------------------------------------------------------------------------------
Model                    Window = 5                   Window = 21                   Window = 63          
                  RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)  
--------------------------------------------------------------------------------------------------------------
GRU             0.0170  0.0112 -1.4292   49.25   0.0296  0.0174 -6.2515   46.62   0.0199  0.0121 -2.2636   54.12  
--------------------------------------------------------------------------------------------------------------


### 1D-CNN

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense
import warnings

warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')
tf.random.set_seed(42)

# 初始化全局存储结构
if 'all_results' not in globals():
    all_results = []
if 'all_errors' not in globals():
    all_errors = []

model_name = "1D-CNN"
current_results = {}

# 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
all_results = [res for res in all_results if res['model'] != model_name]
all_errors = [err for err in all_errors if err['model'] != model_name]

for w, data in experiment_data_refined.items():
    # 1. 提取3D特征张量 (N, L, D) 适配 1D-CNN
    X_train_3d = data['X_train']
    X_test_3d = data['X_test']
    
    y_train = data['y_train'].flatten()
    y_true = data['y_test'].flatten()
    
    N_tr, L, D = X_train_3d.shape
    
    # 2. 模型构建与拟合
    # 根据窗口大小动态调整 kernel_size，防止 kernel_size 大于窗口长度
    k_size = 3 if L >= 3 else L 
    
    model = Sequential([
        Conv1D(filters=32, kernel_size=k_size, activation='relu', input_shape=(L, D)),
        Flatten(),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    
    model.fit(X_train_3d, y_train, epochs=50, batch_size=32, verbose=0, shuffle=False)
    
    # 3. 模型预测
    y_pred = model.predict(X_test_3d, verbose=0).flatten()
    
    # 4. 基准预测 (零收益)
    y_pred_baseline = np.zeros_like(y_true)
    
    # 5. 指标计算
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae = np.mean(np.abs(y_true - y_pred))
    r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
    da = np.mean(np.sign(y_true) == np.sign(y_pred)) * 100
    
    # 6. 误差序列计算
    e_t = y_true - y_pred
    e0_t = y_true - y_pred_baseline
    
    # 7. 组装结果字典
    results_dict = {
        "model": model_name,
        "window": w,
        "RMSE": rmse,
        "MAE": mae,
        "R2_OS": r2_os,
        "DA": da
    }
    
    error_dict = {
        "model": model_name,
        "window": w,
        "errors": e_t,
        "benchmark_errors": e0_t
    }
    
    # 8. 存储结果
    all_results.append(results_dict)
    all_errors.append(error_dict)
    current_results[w] = results_dict

# ---------------- 输出格式模拟 ----------------
print("-" * 110)
# 表头第一行：窗口
header_win = f"{'Model':<15}"
# 表头第二行：指标
header_met = f"{'':<15}"
# 数据行
row_data = f"{model_name:<15}"

for w in experiment_data_refined.keys():
    header_win += f"{f'Window = {w}':^30}"
    header_met += f"{'RMSE':>7} {'MAE':>7} {'R2_OS':>7} {'DA (%)':>7}  "
    
    res = current_results[w]
    row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

print(header_win)
print(header_met)
print("-" * 110)
print(row_data)
print("-" * 110)

--------------------------------------------------------------------------------------------------------------
Model                    Window = 5                   Window = 21                   Window = 63          
                  RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)  
--------------------------------------------------------------------------------------------------------------
1D-CNN          0.0190  0.0146 -2.0158   50.00   0.0205  0.0161 -2.4732   48.80   0.0410  0.0315 -12.8136   48.05  
--------------------------------------------------------------------------------------------------------------


### PatchTST

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Embedding, MultiHeadAttention, LayerNormalization, Dense, Flatten
import warnings

warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')
tf.random.set_seed(42)

# 初始化全局存储结构
if 'all_results' not in globals():
    all_results = []
if 'all_errors' not in globals():
    all_errors = []

model_name = "PatchTST"
current_results = {}

# 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
all_results = [res for res in all_results if res['model'] != model_name]
all_errors = [err for err in all_errors if err['model'] != model_name]

for w, data in experiment_data_refined.items():
    # 1. 提取3D特征张量 (N, L, D)
    X_train_3d = data['X_train']
    X_test_3d = data['X_test']
    
    y_train = data['y_train'].flatten()
    y_true = data['y_test'].flatten()
    
    N_tr, L, D = X_train_3d.shape
    
    # 2. 模型构建与拟合 (使用 Functional API 极简实现 PatchTST 核心逻辑)
    # 动态设定 Patch 长度，确保不同窗口 L 下都能有效分块
    patch_len = max(2, L // 5) 
    
    inputs = Input(shape=(L, D))
    # Patching 阶段：利用 stride Conv1D 实现无重叠切块与线性映射
    x = Conv1D(filters=32, kernel_size=patch_len, strides=patch_len, padding='valid')(inputs)
    
    # Positional Encoding 阶段
    num_patches = x.shape[1]
    positions = tf.range(start=0, limit=num_patches, delta=1)
    pos_emb = Embedding(input_dim=num_patches, output_dim=32)(positions)
    x = x + pos_emb
    
    # Transformer Encoder 阶段 (单层极简版)
    attn_out = MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
    x = LayerNormalization()(x + attn_out)
    ffn_out = Dense(32, activation='relu')(x)
    x = LayerNormalization()(x + ffn_out)
    
    # 预测头
    x = Flatten()(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')
    
    model.fit(X_train_3d, y_train, epochs=50, batch_size=32, verbose=0, shuffle=False)
    
    # 3. 模型预测
    y_pred = model.predict(X_test_3d, verbose=0).flatten()
    
    # 4. 基准预测 (零收益)
    y_pred_baseline = np.zeros_like(y_true)
    
    # 5. 指标计算
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae = np.mean(np.abs(y_true - y_pred))
    r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
    da = np.mean(np.sign(y_true) == np.sign(y_pred)) * 100
    
    # 6. 误差序列计算
    e_t = y_true - y_pred
    e0_t = y_true - y_pred_baseline
    
    # 7. 组装结果字典
    results_dict = {
        "model": model_name,
        "window": w,
        "RMSE": rmse,
        "MAE": mae,
        "R2_OS": r2_os,
        "DA": da
    }
    
    error_dict = {
        "model": model_name,
        "window": w,
        "errors": e_t,
        "benchmark_errors": e0_t
    }
    
    # 8. 存储结果
    all_results.append(results_dict)
    all_errors.append(error_dict)
    current_results[w] = results_dict

# ---------------- 输出格式模拟 ----------------
print("-" * 110)
# 表头第一行：窗口
header_win = f"{'Model':<15}"
# 表头第二行：指标
header_met = f"{'':<15}"
# 数据行
row_data = f"{model_name:<15}"

for w in experiment_data_refined.keys():
    header_win += f"{f'Window = {w}':^30}"
    header_met += f"{'RMSE':>7} {'MAE':>7} {'R2_OS':>7} {'DA (%)':>7}  "
    
    res = current_results[w]
    row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

print(header_win)
print(header_met)
print("-" * 110)
print(row_data)
print("-" * 110)

--------------------------------------------------------------------------------------------------------------
Model                    Window = 5                   Window = 21                   Window = 63          
                  RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)  
--------------------------------------------------------------------------------------------------------------
PatchTST        0.0309  0.0234 -6.9975   50.75   0.0279  0.0185 -5.4267   45.96   0.0458  0.0312 -16.2883   50.34  
--------------------------------------------------------------------------------------------------------------


### iTransformer

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, MultiHeadAttention, LayerNormalization, Permute
import warnings

warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')
tf.random.set_seed(42)

# 初始化全局存储结构
if 'all_results' not in globals():
    all_results = []
if 'all_errors' not in globals():
    all_errors = []

model_name = "iTransformer"
current_results = {}

# 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
all_results = [res for res in all_results if res['model'] != model_name]
all_errors = [err for err in all_errors if err['model'] != model_name]

for w, data in experiment_data_refined.items():
    # 1. 提取3D特征张量 (N, L, D)
    X_train_3d = data['X_train']
    X_test_3d = data['X_test']
    
    y_train = data['y_train'].flatten()
    y_true = data['y_test'].flatten()
    
    N_tr, L, D = X_train_3d.shape
    
    # 2. 模型构建与拟合 (使用 Functional API 实现 iTransformer 核心逻辑)
    inputs = Input(shape=(L, D))
    
    # 修正：使用 Keras 原生 Permute 层代替 tf.transpose 避免 KerasTensor 类型报错
    # 参数 (2, 1) 表示转置输入的第2维度(D)和第1维度(L)，Batch维度0由Keras自动处理隐藏
    x = Permute((2, 1))(inputs)
    
    # 线性映射投影到 d_model 维度
    x = Dense(32)(x)
    
    # Transformer Encoder 阶段 (处理各个变量 Token 之间的交互)
    attn_out = MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
    x = LayerNormalization()(x + attn_out)
    ffn_out = Dense(32, activation='relu')(x)
    x = LayerNormalization()(x + ffn_out)
    
    # 预测头
    x = Flatten()(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')
    
    model.fit(X_train_3d, y_train, epochs=50, batch_size=32, verbose=0, shuffle=False)
    
    # 3. 模型预测
    y_pred = model.predict(X_test_3d, verbose=0).flatten()
    
    # 4. 基准预测 (零收益)
    y_pred_baseline = np.zeros_like(y_true)
    
    # 5. 指标计算
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae = np.mean(np.abs(y_true - y_pred))
    r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
    da = np.mean(np.sign(y_true) == np.sign(y_pred)) * 100
    
    # 6. 误差序列计算
    e_t = y_true - y_pred
    e0_t = y_true - y_pred_baseline
    
    # 7. 组装结果字典
    results_dict = {
        "model": model_name,
        "window": w,
        "RMSE": rmse,
        "MAE": mae,
        "R2_OS": r2_os,
        "DA": da
    }
    
    error_dict = {
        "model": model_name,
        "window": w,
        "errors": e_t,
        "benchmark_errors": e0_t
    }
    
    # 8. 存储结果
    all_results.append(results_dict)
    all_errors.append(error_dict)
    current_results[w] = results_dict

# ---------------- 输出格式模拟 ----------------
print("-" * 110)
# 表头第一行：窗口
header_win = f"{'Model':<15}"
# 表头第二行：指标
header_met = f"{'':<15}"
# 数据行
row_data = f"{model_name:<15}"

for w in experiment_data_refined.keys():
    header_win += f"{f'Window = {w}':^30}"
    header_met += f"{'RMSE':>7} {'MAE':>7} {'R2_OS':>7} {'DA (%)':>7}  "
    
    res = current_results[w]
    row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

print(header_win)
print(header_met)
print("-" * 110)
print(row_data)
print("-" * 110)

--------------------------------------------------------------------------------------------------------------
Model                    Window = 5                   Window = 21                   Window = 63          
                  RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)  
--------------------------------------------------------------------------------------------------------------
iTransformer    0.0378  0.0290 -10.9864   48.28   0.0432  0.0343 -14.4362   48.91   0.0281  0.0219 -5.5111   53.32  
--------------------------------------------------------------------------------------------------------------


### TimesNet-GRU-MHA

In [10]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Conv1D, GRU, MultiHeadAttention, LayerNormalization, Flatten, Dense
# import warnings

# warnings.filterwarnings("ignore")
# tf.get_logger().setLevel('ERROR')
# tf.random.set_seed(42)

# # 初始化全局存储结构
# if 'all_results' not in globals():
#     all_results = []
# if 'all_errors' not in globals():
#     all_errors = []

# model_name = "TimesNet-GRU-MHA"
# current_results = {}

# # 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
# all_results = [res for res in all_results if res['model'] != model_name]
# all_errors = [err for err in all_errors if err['model'] != model_name]

# for w, data in experiment_data_refined.items():
#     # 1. 提取3D特征张量 (N, L, D)
#     X_train_3d = data['X_train']
#     X_test_3d = data['X_test']
    
#     y_train = data['y_train'].flatten()
#     y_true = data['y_test'].flatten()
    
#     N_tr, L, D = X_train_3d.shape
    
#     # 2. 模型构建与拟合 (使用 Functional API 实现级联混合架构)
#     inputs = Input(shape=(L, D))
    
#     # 模块 A: 极简版 TimesNet 核心思想近似 (使用1D卷积捕捉局部多周期时序特征)
#     k_size = 3 if L >= 3 else L 
#     x = Conv1D(filters=32, kernel_size=k_size, padding='same', activation='relu')(inputs)
    
#     # 模块 B: GRU 层 (提取序列全局长短期依赖，保留时间步维度)
#     x = GRU(32, return_sequences=True)(x)
    
#     # 模块 C: MHA 层 (多头注意力机制进行特征加权与跨时间步交互)
#     attn_out = MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
#     x = LayerNormalization()(x + attn_out)
    
#     # 预测头
#     x = Flatten()(x)
#     outputs = Dense(1)(x)
    
#     model = Model(inputs=inputs, outputs=outputs)
#     model.compile(optimizer='adam', loss='mse')
    
#     model.fit(X_train_3d, y_train, epochs=50, batch_size=32, verbose=0, shuffle=False)
    
#     # 3. 模型预测
#     y_pred = model.predict(X_test_3d, verbose=0).flatten()
    
#     # 4. 基准预测 (零收益)
#     y_pred_baseline = np.zeros_like(y_true)
    
#     # 5. 指标计算
#     rmse = np.sqrt(np.mean((y_true - y_pred)**2))
#     mae = np.mean(np.abs(y_true - y_pred))
#     r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
#     da = np.mean(np.sign(y_true) == np.sign(y_pred)) * 100
    
#     # 6. 误差序列计算
#     e_t = y_true - y_pred
#     e0_t = y_true - y_pred_baseline
    
#     # 7. 组装结果字典
#     results_dict = {
#         "model": model_name,
#         "window": w,
#         "RMSE": rmse,
#         "MAE": mae,
#         "R2_OS": r2_os,
#         "DA": da
#     }
    
#     error_dict = {
#         "model": model_name,
#         "window": w,
#         "errors": e_t,
#         "benchmark_errors": e0_t
#     }
    
#     # 8. 存储结果
#     all_results.append(results_dict)
#     all_errors.append(error_dict)
#     current_results[w] = results_dict

# # ---------------- 输出格式模拟 ----------------
# print("-" * 110)
# # 表头第一行：窗口
# header_win = f"{'Model':<20}"
# # 表头第二行：指标
# header_met = f"{'':<20}"
# # 数据行
# row_data = f"{model_name:<20}"

# for w in experiment_data_refined.keys():
#     header_win += f"{f'Window = {w}':^30}"
#     header_met += f"{'RMSE':>7} {'MAE':>7} {'R2_OS':>7} {'DA (%)':>7}  "
    
#     res = current_results[w]
#     row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

# print(header_win)
# print(header_met)
# print("-" * 110)
# print(row_data)
# print("-" * 110)

In [11]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Conv1D, GRU, MultiHeadAttention, LayerNormalization, Dense, Concatenate, Dropout, SpatialDropout1D, GlobalAveragePooling1D
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras import regularizers
# import warnings

# warnings.filterwarnings("ignore")
# tf.get_logger().setLevel('ERROR')
# tf.random.set_seed(42)

# # 初始化全局存储结构
# if 'all_results' not in globals():
#     all_results = []
# if 'all_errors' not in globals():
#     all_errors = []

# model_name = "TimesNet-GRU-MHA"
# current_results = {}

# # 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
# all_results = [res for res in all_results if res['model'] != model_name]
# all_errors = [err for err in all_errors if err['model'] != model_name]

# for w, data in experiment_data_refined.items():
#     # 1. 提取3D特征张量 (N, L, D)
#     X_train_3d = data['X_train']
#     X_test_3d = data['X_test']
    
#     y_train = data['y_train'].flatten()
#     y_true = data['y_test'].flatten()
    
#     N_tr, L, D = X_train_3d.shape
    
#     # 2. 模型构建与拟合 (优化版架构)
#     inputs = Input(shape=(L, D))
    
#     # 模块 A: 多尺度 1D 卷积 (捕获不同频带局部特征)，加入 L2 正则防过拟合
#     k_sizes = list(set([min(k, L) for k in [2, 3, 5]]))
#     convs = [Conv1D(filters=16, kernel_size=k, padding='same', activation='relu', 
#                     kernel_regularizer=regularizers.l2(1e-4))(inputs) for k in k_sizes]
#     x = Concatenate()(convs) if len(convs) > 1 else convs[0]
#     x = SpatialDropout1D(0.2)(x)
    
#     # 模块 B: GRU 层 (降低隐藏单元数防过拟合，保留时间步)
#     x_gru = GRU(32, return_sequences=True)(x)
#     x_gru = Dropout(0.2)(x_gru)
    
#     # 模块 C: MHA 层 (优化注意力头数与维度，引入残差与层归一化)
#     attn_out = MultiHeadAttention(num_heads=2, key_dim=16, dropout=0.2)(x_gru, x_gru)
#     x = LayerNormalization()(x_gru + attn_out)
    
#     # 预测头 (使用全局平均池化替代 Flatten 减少参数量，降低过拟合风险)
#     x = GlobalAveragePooling1D()(x)
#     outputs = Dense(1, kernel_regularizer=regularizers.l2(1e-4))(x)
    
#     model = Model(inputs=inputs, outputs=outputs)
    
#     # 模块 D: 训练策略优化 (动态学习率 + 提前停止)
#     optimizer = Adam(learning_rate=0.001)
#     model.compile(optimizer=optimizer, loss='mse')
    
#     callbacks = [
#         EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
#         ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
#     ]
    
#     # 使用 validation_split=0.1 监控过拟合，提高 epochs 让 ES 自动截断
#     model.fit(X_train_3d, y_train, epochs=150, batch_size=32, 
#               validation_split=0.1, verbose=0, shuffle=False, callbacks=callbacks)
    
#     # 3. 模型预测
#     y_pred = model.predict(X_test_3d, verbose=0).flatten()
    
#     # 4. 基准预测 (零收益)
#     y_pred_baseline = np.zeros_like(y_true)
    
#     # 5. 指标计算
#     rmse = np.sqrt(np.mean((y_true - y_pred)**2))
#     mae = np.mean(np.abs(y_true - y_pred))
#     r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
#     da = np.mean(np.sign(y_true) == np.sign(y_pred)) * 100
    
#     # 6. 误差序列计算
#     e_t = y_true - y_pred
#     e0_t = y_true - y_pred_baseline
    
#     # 7. 组装结果字典
#     results_dict = {
#         "model": model_name,
#         "window": w,
#         "RMSE": rmse,
#         "MAE": mae,
#         "R2_OS": r2_os,
#         "DA": da
#     }
    
#     error_dict = {
#         "model": model_name,
#         "window": w,
#         "errors": e_t,
#         "benchmark_errors": e0_t
#     }
    
#     # 8. 存储结果
#     all_results.append(results_dict)
#     all_errors.append(error_dict)
#     current_results[w] = results_dict

# # ---------------- 输出格式模拟 ----------------
# print("-" * 110)
# # 表头第一行：窗口
# header_win = f"{'Model':<20}"
# # 表头第二行：指标
# header_met = f"{'':<20}"
# # 数据行
# row_data = f"{model_name:<20}"

# for w in experiment_data_refined.keys():
#     header_win += f"{f'Window = {w}':^30}"
#     header_met += f"{'RMSE':>7} {'MAE':>7} {'R2_OS':>7} {'DA (%)':>7}  "
    
#     res = current_results[w]
#     row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

# print(header_win)
# print(header_met)
# print("-" * 110)
# print(row_data)
# print("-" * 110)

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GRU, MultiHeadAttention, LayerNormalization, Dense, Concatenate, Dropout, SpatialDropout1D, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import warnings

warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')
tf.random.set_seed(42)

# 初始化全局存储结构
if 'all_results' not in globals():
    all_results = []
if 'all_errors' not in globals():
    all_errors = []

model_name = "TimesNet-GRU-MHA"
current_results = {}

# 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
all_results = [res for res in all_results if res['model'] != model_name]
all_errors = [err for err in all_errors if err['model'] != model_name]

for w, data in experiment_data_refined.items():
    # 1. 提取3D特征张量 (N, L, D)
    X_train_3d = data['X_train']
    X_test_3d = data['X_test']
    
    y_train = data['y_train'].flatten()
    y_true = data['y_test'].flatten()
    
    N_tr, L, D = X_train_3d.shape
    
    # 2. 模型构建与拟合 (优化版架构)
    inputs = Input(shape=(L, D))
    
    # 模块 A: 多尺度 1D 卷积 (捕获不同频带局部特征)，加入 L2 正则防过拟合
    k_sizes = list(set([min(k, L) for k in [2, 3, 5]]))
    convs = [Conv1D(filters=16, kernel_size=k, padding='same', activation='relu', 
                    kernel_regularizer=regularizers.l2(1e-4))(inputs) for k in k_sizes]
    x = Concatenate()(convs) if len(convs) > 1 else convs[0]
    x = SpatialDropout1D(0.2)(x)
    
    # 模块 B: GRU 层 (降低隐藏单元数防过拟合，保留时间步)
    x_gru = GRU(24, return_sequences=True)(x)
    x_gru = Dropout(0.2)(x_gru)
    
    # 模块 C: MHA 层 (优化注意力头数与维度，引入残差与层归一化)
    attn_out = MultiHeadAttention(num_heads=5, key_dim=16, dropout=0.2)(x_gru, x_gru)
    x = LayerNormalization()(x_gru + attn_out)
    
    # 预测头 (使用全局平均池化替代 Flatten 减少参数量，降低过拟合风险)
    x = GlobalAveragePooling1D()(x)
    outputs = Dense(1, kernel_regularizer=regularizers.l2(1e-4))(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    # 模块 D: 训练策略优化 (动态学习率 + 提前停止)
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
    ]
    
    # 使用 validation_split=0.1 监控过拟合，提高 epochs 让 ES 自动截断
    model.fit(X_train_3d, y_train, epochs=150, batch_size=32, 
              validation_split=0.1, verbose=0, shuffle=False, callbacks=callbacks)
    
    # 3. 模型预测
    y_pred = model.predict(X_test_3d, verbose=0).flatten()
    
    # 4. 基准预测 (零收益)
    y_pred_baseline = np.zeros_like(y_true)
    
    # 5. 指标计算
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae = np.mean(np.abs(y_true - y_pred))
    r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
    da = np.mean(np.sign(y_true) == np.sign(y_pred)) * 100
    
    # 6. 误差序列计算
    e_t = y_true - y_pred
    e0_t = y_true - y_pred_baseline
    
    # 7. 组装结果字典
    results_dict = {
        "model": model_name,
        "window": w,
        "RMSE": rmse,
        "MAE": mae,
        "R2_OS": r2_os,
        "DA": da
    }
    
    error_dict = {
        "model": model_name,
        "window": w,
        "errors": e_t,
        "benchmark_errors": e0_t
    }
    
    # 8. 存储结果
    all_results.append(results_dict)
    all_errors.append(error_dict)
    current_results[w] = results_dict

# ---------------- 输出格式模拟 ----------------
print("-" * 110)
# 表头第一行：窗口
header_win = f"{'Model':<20}"
# 表头第二行：指标
header_met = f"{'':<20}"
# 数据行
row_data = f"{model_name:<20}"

for w in experiment_data_refined.keys():
    header_win += f"{f'Window = {w}':^30}"
    header_met += f"{'RMSE':>7} {'MAE':>7} {'R2_OS':>7} {'DA (%)':>7}  "
    
    res = current_results[w]
    row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

print(header_win)
print(header_met)
print("-" * 110)
print(row_data)
print("-" * 110)

--------------------------------------------------------------------------------------------------------------
Model                         Window = 5                   Window = 21                   Window = 63          
                       RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)  
--------------------------------------------------------------------------------------------------------------
TimesNet-GRU-MHA     0.0110  0.0080 -0.0097   48.71   0.0110  0.0080 -0.0024   50.33   0.0110  0.0081 -0.0014   51.49  
--------------------------------------------------------------------------------------------------------------


### 基准DM检验

In [61]:
import numpy as np
import scipy.stats as stats
import warnings

warnings.filterwarnings("ignore")

# 1. 定义 Diebold-Mariano 检验函数
def dm_test(e1, e2, h=1):
    """
    e1: 目标模型误差 (TimesNet-GRU-MHA)
    e2: 基准模型误差 (Baseline)
    h: 预测步长 (默认为 1)
    """
    # ====== [安全机制：强制尾部对齐] ======
    min_len = min(len(e1), len(e2))
    e1 = np.array(e1)[-min_len:]
    e2 = np.array(e2)[-min_len:]
    # ======================================
    
    d = e2**2 - e1**2 
    T = len(d)
    d_mean = np.mean(d)
    
    # 计算自动协方差 (Newey-West)
    gamma = np.zeros(h)
    for i in range(h):
        gamma[i] = np.sum((d[i:] - d_mean) * (d[:T-i] - d_mean)) / T
        
    var_d = gamma[0]
    for i in range(1, h):
        var_d += 2.0 * gamma[i]
        
    if var_d <= 0:
        return np.nan, np.nan
        
    DM_stat = d_mean / np.sqrt(var_d / T)
    p_value = 2 * (1 - stats.norm.cdf(abs(DM_stat)))
    
    return DM_stat, p_value

# ====== [修复点：增加 stat 参数，严格限制只奖励正向效果] ======
def get_significance_stars(p, stat):
    if np.isnan(p) or stat <= 0: return ""
    if p < 0.01: return "***"
    if p < 0.05: return "**"
    if p < 0.10: return "*"
    return ""
# ==============================================================

# 2. 提取需要检验的模型名称与动态获取窗口列表
target_model = "TimesNet-GRU-MHA"
baselines = ["ARIMAX", "XGBoost", "LSTM", "GRU", "1D-CNN", "PatchTST", "iTransformer"]

# 从全局字典中动态获取所有存在的窗口
windows = sorted(list(set([err['window'] for err in all_errors])))

dm_results = {b: {} for b in baselines}

# 3. 遍历计算目标模型与各个基准模型在不同窗口下的 DM 统计量
for w in windows:
    # 获取目标模型在当前窗口的最新误差序列
    e_target_list = [err['errors'] for err in all_errors if err['model'] == target_model and err['window'] == w]
    if not e_target_list:
        print(f"警告: 未找到目标模型 {target_model} 在 Window={w} 的误差序列。")
        continue
    e_target = e_target_list[-1] 
    
    for b in baselines:
        e_base_list = [err['errors'] for err in all_errors if err['model'] == b and err['window'] == w]
        if not e_base_list:
            dm_results[b][w] = (np.nan, np.nan)
            continue
        e_base = e_base_list[-1]
        
        # 计算 DM 统计量与 p 值
        stat, p = dm_test(e_target, e_base, h=1)
        dm_results[b][w] = (stat, p)

In [65]:
# ---------------- 4. 输出格式 (Diebold-Mariano Test Results) ----------------
print("-" * 120)
header_win = f"{'Baseline Model':<20}"
header_met = f"{'':<20}"

for w in windows:
    header_win += f"{f'Window = {w}':^32}"
    header_met += f"{'DM-Stat':>10} {'p-value':>10} {'Sig':>8}  "

print(header_win)
print(header_met)
print("-" * 120)

for b in baselines:
    row = f"{b:<20}"
    for w in windows:
        if w in dm_results[b] and not np.isnan(dm_results[b][w][0]):
            stat, p = dm_results[b][w]
            # 这里传入 stat 配合我们修复的函数
            stars = get_significance_stars(p, stat)
            row += f"{stat:>10.6f} {p:>10.6f} {stars:>8}  "
        else:
            row += f"{'-':>10} {'-':>10} {'':>8}  "
    print(row)
print("-" * 120)
print(f"Note: Target Model = {target_model}.")
print("Positive DM-Stat indicates the Target Model outperforms the Baseline Model (MSE loss).")
print("Significance levels: *** p<0.01, ** p<0.05, * p<0.1 (Only when Target > Baseline).")

------------------------------------------------------------------------------------------------------------------------
Baseline Model                 Window = 5                     Window = 21                     Window = 63           
                       DM-Stat    p-value      Sig     DM-Stat    p-value      Sig     DM-Stat    p-value      Sig  
------------------------------------------------------------------------------------------------------------------------
ARIMAX                3.007523   0.002634      ***    4.718623   0.000002      ***    8.366116   0.000000      ***  
XGBoost               6.180784   0.000000      ***    6.923504   0.000000      ***    6.559726   0.000000      ***  
LSTM                  3.448898   0.000563      ***    5.440346   0.000000      ***    6.982010   0.000000      ***  
GRU                   2.873298   0.004062      ***    2.654235   0.007949      ***    2.802451   0.005072      ***  
1D-CNN               12.609740   0.000000      ***   14.

基准模型 (Baselines) 导出

In [97]:
import pandas as pd
import os

# 提取并扁平化嵌套字典数据
rows = []
for model_name, window_data in dm_results.items():
    for w, (stat, p) in window_data.items():
        stars = get_significance_stars(p, stat) if not np.isnan(stat) else ""
        rows.append({
            "Model": model_name,
            "Window": w,
            "DM_Stat": stat,
            "P_Value": p,
            "Significance": stars
        })

df_dm = pd.DataFrame(rows)

# 直接建目录并保存 CSV
os.makedirs('../results/tables', exist_ok=True)
save_path = '../results/tables/dm_results_baselines.csv'
df_dm.to_csv(save_path, index=False)

print(f"The DM results of Baselines have been saved to: {save_path}")

The DM results of Baselines have been saved to: ../results/tables/dm_results_baselines.csv


## 消融实验

### w/o FFT Extraction（移除动态周期发现）

In [15]:
# 消融实验版本：TimesNet-GRU-MHA (w/o FFT Extraction)
# 移除多尺度自适应提取，强制使用固定周期 (金融常见 p=5 天) 将一维折叠为二维进行特征提取

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GRU, MultiHeadAttention, LayerNormalization, Dense, Concatenate, Dropout, SpatialDropout1D, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import warnings

warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')
tf.random.set_seed(42)

# 初始化全局存储结构
if 'all_results' not in globals():
    all_results = []
if 'all_errors' not in globals():
    all_errors = []

# 消融版本名称
model_name = "TimesNet-GRU-MHA (w/o FFT)"
current_results = {}

# 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
all_results = [res for res in all_results if res['model'] != model_name]
all_errors = [err for err in all_errors if err['model'] != model_name]

for w, data in experiment_data_refined.items():
    # 1. 提取3D特征张量 (N, L, D)
    X_train_3d = data['X_train']
    X_test_3d = data['X_test']
    
    y_train = data['y_train'].flatten()
    y_true = data['y_test'].flatten()
    
    N_tr, L, D = X_train_3d.shape
    
    # 2. 模型构建与拟合
    inputs = Input(shape=(L, D))
    
    # 【消融点】：移除模块 A (多尺度 1D 卷积/动态周期发现)，直接将特征送入模块 B
    
    # 模块 B: GRU 层 (降低隐藏单元数防过拟合，保留时间步)
    x_gru = GRU(24, return_sequences=True)(inputs)
    x_gru = Dropout(0.2)(x_gru)
    
    # 模块 C: MHA 层 (优化注意力头数与维度，引入残差与层归一化)
    attn_out = MultiHeadAttention(num_heads=5, key_dim=16, dropout=0.2)(x_gru, x_gru)
    x = LayerNormalization()(x_gru + attn_out)
    
    # 预测头 (使用全局平均池化替代 Flatten 减少参数量，降低过拟合风险)
    x = GlobalAveragePooling1D()(x)
    outputs = Dense(1, kernel_regularizer=regularizers.l2(1e-4))(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    # 模块 D: 训练策略优化 (动态学习率 + 提前停止)
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
    ]
    
    # 使用 validation_split=0.1 监控过拟合，提高 epochs 让 ES 自动截断
    model.fit(X_train_3d, y_train, epochs=150, batch_size=32, 
              validation_split=0.1, verbose=0, shuffle=False, callbacks=callbacks)
    
    # 3. 模型预测
    y_pred = model.predict(X_test_3d, verbose=0).flatten()
    
    # 4. 基准预测 (零收益)
    y_pred_baseline = np.zeros_like(y_true)
    
    # 5. 指标计算
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae = np.mean(np.abs(y_true - y_pred))
    r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
    da = np.mean(np.sign(y_true) == np.sign(y_pred)) * 100
    
    # 6. 误差序列计算
    e_t = y_true - y_pred
    e0_t = y_true - y_pred_baseline
    
    # 7. 组装结果字典
    results_dict = {
        "model": model_name,
        "window": w,
        "RMSE": rmse,
        "MAE": mae,
        "R2_OS": r2_os,
        "DA": da
    }
    
    error_dict = {
        "model": model_name,
        "window": w,
        "errors": e_t,
        "benchmark_errors": e0_t
    }
    
    # 8. 存储结果
    all_results.append(results_dict)
    all_errors.append(error_dict)
    current_results[w] = results_dict

# ---------------- 输出格式模拟 ----------------
print("-" * 110)
# 表头第一行：窗口
header_win = f"{'Model':<20}"
# 表头第二行：指标
header_met = f"{'':<20}"
# 数据行
row_data = f"{model_name:<20}"

for w in experiment_data_refined.keys():
    header_win += f"{f'Window = {w}':^30}"
    header_met += f"{'RMSE':>7} {'MAE':>7} {'R2_OS':>7} {'DA (%)':>7}  "
    
    res = current_results[w]
    row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

print(header_win)
print(header_met)
print("-" * 110)
print(row_data)
print("-" * 110)

--------------------------------------------------------------------------------------------------------------
Model                         Window = 5                   Window = 21                   Window = 63          
                       RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)  
--------------------------------------------------------------------------------------------------------------
TimesNet-GRU-MHA (w/o FFT) 0.0116  0.0085 -0.1283   47.64   0.0176  0.0116 -1.5573   47.93   0.0324  0.0268 -7.6433   49.43  
--------------------------------------------------------------------------------------------------------------


### w/o 2D Inception（退化为 1D 基线）

In [93]:
# 消融实验版本：TimesNet-GRU-MHA (w/o 2D Inception)
# 移除 1D 到 2D 重塑及多尺度 Inception 结构，退化为标准的单尺度 1D 卷积表示

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GRU, MultiHeadAttention, LayerNormalization, Dense, Concatenate, Dropout, SpatialDropout1D, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import warnings

warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')
tf.random.set_seed(42)

# 初始化全局存储结构
if 'all_results' not in globals():
    all_results = []
if 'all_errors' not in globals():
    all_errors = []

# 消融版本名称
model_name = "TimesNet-GRU-MHA (w/o 2D Inception)"
current_results = {}

# 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
all_results = [res for res in all_results if res['model'] != model_name]
all_errors = [err for err in all_errors if err['model'] != model_name]

for w, data in experiment_data_refined.items():
    # 1. 提取3D特征张量 (N, L, D)
    X_train_3d = data['X_train']
    X_test_3d = data['X_test']
    
    y_train = data['y_train'].flatten()
    y_true = data['y_test'].flatten()
    
    N_tr, L, D = X_train_3d.shape
    
    # 2. 模型构建与拟合
    inputs = Input(shape=(L, D))
    
    # 【消融点】：退化为标准的单尺度 1D 卷积表示 (移除多尺度/Inception特性)
    x = Conv1D(filters=16, kernel_size=3, padding='same', activation='relu', 
               kernel_regularizer=regularizers.l2(1e-4))(inputs)
    x = SpatialDropout1D(0.1)(x)
    
    # 模块 B: GRU 层 (降低隐藏单元数防过拟合，保留时间步)
    x_gru = GRU(24, return_sequences=True)(x)
    x_gru = Dropout(0.1)(x_gru)
    
    # 模块 C: MHA 层 (优化注意力头数与维度，引入残差与层归一化)
    attn_out = MultiHeadAttention(num_heads=5, key_dim=16, dropout=0.1)(x_gru, x_gru)
    x = LayerNormalization()(x_gru + attn_out)
    
    # 预测头 (使用全局平均池化替代 Flatten 减少参数量，降低过拟合风险)
    x = GlobalAveragePooling1D()(x)
    outputs = Dense(1, kernel_regularizer=regularizers.l2(1e-4))(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    # 模块 D: 训练策略优化 (动态学习率 + 提前停止)
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
    ]
    
    # 使用 validation_split=0.1 监控过拟合，提高 epochs 让 ES 自动截断
    model.fit(X_train_3d, y_train, epochs=150, batch_size=32, 
              validation_split=0.1, verbose=0, shuffle=False, callbacks=callbacks)
    
    # 3. 模型预测
    y_pred = model.predict(X_test_3d, verbose=0).flatten()
    
    # 4. 基准预测 (零收益)
    y_pred_baseline = np.zeros_like(y_true)
    
    # 5. 指标计算
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae = np.mean(np.abs(y_true - y_pred))
    r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
    da = np.mean(np.sign(y_true) == np.sign(y_pred)) * 100
    
    # 6. 误差序列计算
    e_t = y_true - y_pred
    e0_t = y_true - y_pred_baseline
    
    # 7. 组装结果字典
    results_dict = {
        "model": model_name,
        "window": w,
        "RMSE": rmse,
        "MAE": mae,
        "R2_OS": r2_os,
        "DA": da
    }
    
    error_dict = {
        "model": model_name,
        "window": w,
        "errors": e_t,
        "benchmark_errors": e0_t
    }
    
    # 8. 存储结果
    all_results.append(results_dict)
    all_errors.append(error_dict)
    current_results[w] = results_dict

# ---------------- 输出格式模拟 ----------------
print("-" * 110)
# 表头第一行：窗口
header_win = f"{'Model':<20}"
# 表头第二行：指标
header_met = f"{'':<20}"
# 数据行
row_data = f"{model_name:<20}"

for w in experiment_data_refined.keys():
    header_win += f"{f'Window = {w}':^30}"
    header_met += f"{'RMSE':>7} {'MAE':>7} {'R2_OS':>7} {'DA (%)':>7}  "
    
    res = current_results[w]
    row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

print(header_win)
print(header_met)
print("-" * 110)
print(row_data)
print("-" * 110)

--------------------------------------------------------------------------------------------------------------
Model                         Window = 5                   Window = 21                   Window = 63          
                       RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)  
--------------------------------------------------------------------------------------------------------------
TimesNet-GRU-MHA (w/o 2D Inception) 0.0114  0.0084 -0.0939   45.39   0.0111  0.0081 -0.0199   48.03   0.0115  0.0086 -0.0874   47.94  
--------------------------------------------------------------------------------------------------------------


### w/o GRU（移除循环非线性演化模块）

In [17]:
# 消融实验版本：w/o GRU (即 TimesNet-MHA，移除 GRU 层)

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GRU, MultiHeadAttention, LayerNormalization, Dense, Concatenate, Dropout, SpatialDropout1D, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import warnings

warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')
tf.random.set_seed(42)

# 初始化全局存储结构
if 'all_results' not in globals():
    all_results = []
if 'all_errors' not in globals():
    all_errors = []

# 消融版本名称
model_name = "TimesNet-MHA (w/o GRU)"
current_results = {}

# 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
all_results = [res for res in all_results if res['model'] != model_name]
all_errors = [err for err in all_errors if err['model'] != model_name]

for w, data in experiment_data_refined.items():
    # 1. 提取3D特征张量 (N, L, D)
    X_train_3d = data['X_train']
    X_test_3d = data['X_test']
    
    y_train = data['y_train'].flatten()
    y_true = data['y_test'].flatten()
    
    N_tr, L, D = X_train_3d.shape
    
    # 2. 模型构建与拟合
    inputs = Input(shape=(L, D))
    
    # 模块 A: 多尺度 1D 卷积 (捕获不同频带局部特征)，加入 L2 正则防过拟合
    k_sizes = list(set([min(k, L) for k in [2, 3, 5]]))
    convs = [Conv1D(filters=16, kernel_size=k, padding='same', activation='relu', 
                    kernel_regularizer=regularizers.l2(1e-4))(inputs) for k in k_sizes]
    x = Concatenate()(convs) if len(convs) > 1 else convs[0]
    x = SpatialDropout1D(0.2)(x)
    
    # 【消融点】：移除模块 B (GRU 层)，特征直接送入 MHA 层
    
    # 模块 C: MHA 层 (优化注意力头数与维度，引入残差与层归一化)
    attn_out = MultiHeadAttention(num_heads=5, key_dim=16, dropout=0.2)(x, x)
    x = LayerNormalization()(x + attn_out)
    
    # 预测头 (使用全局平均池化替代 Flatten 减少参数量，降低过拟合风险)
    x = GlobalAveragePooling1D()(x)
    outputs = Dense(1, kernel_regularizer=regularizers.l2(1e-4))(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    # 模块 D: 训练策略优化 (动态学习率 + 提前停止)
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
    ]
    
    # 使用 validation_split=0.1 监控过拟合，提高 epochs 让 ES 自动截断
    model.fit(X_train_3d, y_train, epochs=150, batch_size=32, 
              validation_split=0.1, verbose=0, shuffle=False, callbacks=callbacks)
    
    # 3. 模型预测
    y_pred = model.predict(X_test_3d, verbose=0).flatten()
    
    # 4. 基准预测 (零收益)
    y_pred_baseline = np.zeros_like(y_true)
    
    # 5. 指标计算
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae = np.mean(np.abs(y_true - y_pred))
    r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
    da = np.mean(np.sign(y_true) == np.sign(y_pred)) * 100
    
    # 6. 误差序列计算
    e_t = y_true - y_pred
    e0_t = y_true - y_pred_baseline
    
    # 7. 组装结果字典
    results_dict = {
        "model": model_name,
        "window": w,
        "RMSE": rmse,
        "MAE": mae,
        "R2_OS": r2_os,
        "DA": da
    }
    
    error_dict = {
        "model": model_name,
        "window": w,
        "errors": e_t,
        "benchmark_errors": e0_t
    }
    
    # 8. 存储结果
    all_results.append(results_dict)
    all_errors.append(error_dict)
    current_results[w] = results_dict

# ---------------- 输出格式模拟 ----------------
print("-" * 110)
# 表头第一行：窗口
header_win = f"{'Model':<20}"
# 表头第二行：指标
header_met = f"{'':<20}"
# 数据行
row_data = f"{model_name:<20}"

for w in experiment_data_refined.keys():
    header_win += f"{f'Window = {w}':^30}"
    header_met += f"{'RMSE':>7} {'MAE':>7} {'R2_OS':>7} {'DA (%)':>7}  "
    
    res = current_results[w]
    row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

print(header_win)
print(header_met)
print("-" * 110)
print(row_data)
print("-" * 110)

--------------------------------------------------------------------------------------------------------------
Model                         Window = 5                   Window = 21                   Window = 63          
                       RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)  
--------------------------------------------------------------------------------------------------------------
TimesNet-MHA (w/o GRU) 0.0111  0.0081 -0.0259   49.14   0.0232  0.0187 -3.4532   48.47   0.0136  0.0107 -0.5168   46.11  
--------------------------------------------------------------------------------------------------------------


### w/o MHA（移除多头注意力机制）

In [90]:
# 消融实验版本：w/o MHA (即 TimesNet-GRU，移除多头注意力机制)

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GRU, MultiHeadAttention, LayerNormalization, Dense, Concatenate, Dropout, SpatialDropout1D, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import warnings

warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')
tf.random.set_seed(42)

# 初始化全局存储结构
if 'all_results' not in globals():
    all_results = []
if 'all_errors' not in globals():
    all_errors = []

# 消融版本名称
model_name = "TimesNet-GRU (w/o MHA)"
current_results = {}

# 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
all_results = [res for res in all_results if res['model'] != model_name]
all_errors = [err for err in all_errors if err['model'] != model_name]

for w, data in experiment_data_refined.items():
    # 1. 提取3D特征张量 (N, L, D)
    X_train_3d = data['X_train']
    X_test_3d = data['X_test']
    
    y_train = data['y_train'].flatten()
    y_true = data['y_test'].flatten()
    
    N_tr, L, D = X_train_3d.shape
    
    # 2. 模型构建与拟合
    inputs = Input(shape=(L, D))
    
    # 模块 A: 多尺度 1D 卷积 (捕获不同频带局部特征)，加入 L2 正则防过拟合
    k_sizes = list(set([min(k, L) for k in [2, 3, 5]]))
    convs = [Conv1D(filters=16, kernel_size=k, padding='same', activation='relu', 
                    kernel_regularizer=regularizers.l2(1e-4))(inputs) for k in k_sizes]
    x = Concatenate()(convs) if len(convs) > 1 else convs[0]
    x = SpatialDropout1D(0.001)(x)
    
    # 模块 B: GRU 层 (降低隐藏单元数防过拟合，保留时间步)
    x_gru = GRU(32, return_sequences=True)(x)
    x_gru = Dropout(0.001)(x_gru)
    
    # 【消融点】：移除模块 C (MHA 层与层归一化残差连接)，GRU 输出直接进入预测头
    
    # 预测头 (使用全局平均池化替代 Flatten 减少参数量，降低过拟合风险)
    x = GlobalAveragePooling1D()(x_gru)
    outputs = Dense(1, kernel_regularizer=regularizers.l2(1e-4))(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    # 模块 D: 训练策略优化 (动态学习率 + 提前停止)
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
    ]
    
    # 使用 validation_split=0.1 监控过拟合，提高 epochs 让 ES 自动截断
    model.fit(X_train_3d, y_train, epochs=150, batch_size=32, 
              validation_split=0.1, verbose=0, shuffle=False, callbacks=callbacks)
    
    # 3. 模型预测
    y_pred = model.predict(X_test_3d, verbose=0).flatten()
    
    # 4. 基准预测 (零收益)
    y_pred_baseline = np.zeros_like(y_true)
    
    # 5. 指标计算
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae = np.mean(np.abs(y_true - y_pred))
    r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
    da = np.mean(np.sign(y_true) == np.sign(y_pred)) * 100
    
    # 6. 误差序列计算
    e_t = y_true - y_pred
    e0_t = y_true - y_pred_baseline
    
    # 7. 组装结果字典
    results_dict = {
        "model": model_name,
        "window": w,
        "RMSE": rmse,
        "MAE": mae,
        "R2_OS": r2_os,
        "DA": da
    }
    
    error_dict = {
        "model": model_name,
        "window": w,
        "errors": e_t,
        "benchmark_errors": e0_t
    }
    
    # 8. 存储结果
    all_results.append(results_dict)
    all_errors.append(error_dict)
    current_results[w] = results_dict

# ---------------- 输出格式模拟 ----------------
print("-" * 110)
# 表头第一行：窗口
header_win = f"{'Model':<20}"
# 表头第二行：指标
header_met = f"{'':<20}"
# 数据行
row_data = f"{model_name:<20}"

for w in experiment_data_refined.keys():
    header_win += f"{f'Window = {w}':^30}"
    header_met += f"{'RMSE':>7} {'MAE':>7} {'R2_OS':>7} {'DA (%)':>7}  "
    
    res = current_results[w]
    row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

print(header_win)
print(header_met)
print("-" * 110)
print(row_data)
print("-" * 110)

--------------------------------------------------------------------------------------------------------------
Model                         Window = 5                   Window = 21                   Window = 63          
                       RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)  
--------------------------------------------------------------------------------------------------------------
TimesNet-GRU (w/o MHA) 0.0114  0.0084 -0.0836   48.50   0.0111  0.0082 -0.0276   47.49   0.0113  0.0084 -0.0443   48.05  
--------------------------------------------------------------------------------------------------------------


### w/o TimesNet (即普通的 GRU-MHA)

In [71]:
# 消融实验版本：w/o TimesNet (即普通的 GRU-MHA，移除多尺度1D卷积模块)

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GRU, MultiHeadAttention, LayerNormalization, Dense, Concatenate, Dropout, SpatialDropout1D, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import warnings

warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')
tf.random.set_seed(42)

# 初始化全局存储结构
if 'all_results' not in globals():
    all_results = []
if 'all_errors' not in globals():
    all_errors = []

# 消融版本名称
model_name = "GRU-MHA (w/o TimesNet)"
current_results = {}

# 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
all_results = [res for res in all_results if res['model'] != model_name]
all_errors = [err for err in all_errors if err['model'] != model_name]

for w, data in experiment_data_refined.items():
    # 1. 提取3D特征张量 (N, L, D)
    X_train_3d = data['X_train']
    X_test_3d = data['X_test']
    
    y_train = data['y_train'].flatten()
    y_true = data['y_test'].flatten()
    
    N_tr, L, D = X_train_3d.shape
    
    # 2. 模型构建与拟合
    inputs = Input(shape=(L, D))
    
    # 【消融点】：移除模块 A (多尺度 1D 卷积/TimesNet表示)，特征直接送入 GRU 层
    
    # 模块 B: GRU 层 (降低隐藏单元数防过拟合，保留时间步)
    x_gru = GRU(24, return_sequences=True)(inputs)
    x_gru = Dropout(0.1)(x_gru)
    
    # 模块 C: MHA 层 (优化注意力头数与维度，引入残差与层归一化)
    attn_out = MultiHeadAttention(num_heads=5, key_dim=16, dropout=0.1)(x_gru, x_gru)
    x = LayerNormalization()(x_gru + attn_out)
    
    # 预测头 (使用全局平均池化替代 Flatten 减少参数量，降低过拟合风险)
    x = GlobalAveragePooling1D()(x)
    outputs = Dense(1, kernel_regularizer=regularizers.l2(1e-4))(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    # 模块 D: 训练策略优化 (动态学习率 + 提前停止)
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
    ]
    
    # 使用 validation_split=0.1 监控过拟合，提高 epochs 让 ES 自动截断
    model.fit(X_train_3d, y_train, epochs=150, batch_size=32, 
              validation_split=0.1, verbose=0, shuffle=False, callbacks=callbacks)
    
    # 3. 模型预测
    y_pred = model.predict(X_test_3d, verbose=0).flatten()
    
    # 4. 基准预测 (零收益)
    y_pred_baseline = np.zeros_like(y_true)
    
    # 5. 指标计算
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae = np.mean(np.abs(y_true - y_pred))
    r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
    da = np.mean(np.sign(y_true) == np.sign(y_pred)) * 100
    
    # 6. 误差序列计算
    e_t = y_true - y_pred
    e0_t = y_true - y_pred_baseline
    
    # 7. 组装结果字典
    results_dict = {
        "model": model_name,
        "window": w,
        "RMSE": rmse,
        "MAE": mae,
        "R2_OS": r2_os,
        "DA": da
    }
    
    error_dict = {
        "model": model_name,
        "window": w,
        "errors": e_t,
        "benchmark_errors": e0_t
    }
    
    # 8. 存储结果
    all_results.append(results_dict)
    all_errors.append(error_dict)
    current_results[w] = results_dict

# ---------------- 输出格式模拟 ----------------
print("-" * 110)
# 表头第一行：窗口
header_win = f"{'Model':<20}"
# 表头第二行：指标
header_met = f"{'':<20}"
# 数据行
row_data = f"{model_name:<20}"

for w in experiment_data_refined.keys():
    header_win += f"{f'Window = {w}':^30}"
    header_met += f"{'RMSE':>7} {'MAE':>7} {'R2_OS':>7} {'DA (%)':>7}  "
    
    res = current_results[w]
    row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

print(header_win)
print(header_met)
print("-" * 110)
print(row_data)
print("-" * 110)

--------------------------------------------------------------------------------------------------------------
Model                         Window = 5                   Window = 21                   Window = 63          
                       RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)     RMSE     MAE   R2_OS  DA (%)  
--------------------------------------------------------------------------------------------------------------
GRU-MHA (w/o TimesNet) 0.0120  0.0088 -0.2115   48.82   0.0135  0.0100 -0.5125   48.91   0.0232  0.0181 -3.4307   48.05  
--------------------------------------------------------------------------------------------------------------


### 消融DM检验

In [94]:
import numpy as np
import scipy.stats as stats
import warnings

warnings.filterwarnings("ignore")

# 1. 定义基于滚动窗口的 Diebold-Mariano (DM) 检验函数 (带 HLN 校正)
def dm_test(e1, e2, h=1, loss_type='MSE'):
    """
    e1: 目标模型误差 (Target Model: TimesNet-GRU-MHA)
    e2: 消融/基准模型误差 (Ablated/Baseline Model)
    h: 预测步长 (Forecast horizon，此处默认为单步 1)
    loss_type: 'MSE' 或 'MAE'
    """
    # [安全机制]：强制尾部对齐，防止不同模型测试集截断长度差异导致 broadcast 报错
    min_len = min(len(e1), len(e2))
    e1 = np.array(e1)[-min_len:]
    e2 = np.array(e2)[-min_len:]
    
    # 计算损失差异序列 d_t = Loss(e2) - Loss(e1)
    # 注意：d_t > 0 说明基准模型(e2)误差更大，目标模型(e1)预测更准
    if loss_type == 'MSE':
        d = e2**2 - e1**2
    elif loss_type == 'MAE':
        d = np.abs(e2) - np.abs(e1)
    else:
        raise ValueError("loss_type 必须是 'MSE' 或 'MAE'")
        
    T = len(d)
    mean_d = np.mean(d)
    
    # 计算自协方差 (Auto-covariance)
    def autocovariance(k):
        if k == 0:
            return np.mean((d - mean_d)**2)
        else:
            return np.sum((d[k:] - mean_d) * (d[:-k] - mean_d)) / T
    
    gamma_0 = autocovariance(0)
    
    # 对于多步预测 (h > 1)，计算 HAC 方差
    var_d = gamma_0
    for k in range(1, h):
        var_d += 2 * autocovariance(k)
        
    if var_d == 0 or np.isnan(var_d):
        return np.nan, np.nan
        
    # 计算基础 DM 统计量
    dm_stat = mean_d / np.sqrt(var_d / T)
    
    # 加入 HLN 小样本偏差校正 (针对金融序列高噪和样本限制优化)
    hln_correction = np.sqrt((T + 1 - 2*h + h*(h-1)/T) / T)
    
    # 将校正系数正确应用到统计量
    dm_stat_hln = dm_stat * hln_correction
    
    # 使用自由度为 T-1 的 Student-t 分布计算双侧 p-value
    p_value = 2 * (1 - stats.t.cdf(np.abs(dm_stat_hln), df=T-1))
    
    return dm_stat_hln, p_value

# 判断显著性星号：只有当统计量 > 0 (即目标模型误差真的比对照组小) 且 p值显著时，才打星号
def get_significance_stars(p, stat):
    if np.isnan(p) or stat <= 0: return ""
    if p < 0.01: return "***"
    if p < 0.05: return "**"
    if p < 0.10: return "*"
    return ""

# 2. 提取需要检验的模型名称与动态获取窗口
target_model = "TimesNet-GRU-MHA"

# 动态获取所有对比模型 (直接从数据里抓取，不用手写)
ablations = [
    "TimesNet-GRU-MHA (w/o FFT)", 
    "TimesNet-GRU-MHA (w/o 2D Inception)", 
    "TimesNet-MHA (w/o GRU)", 
    "TimesNet-GRU (w/o MHA)",
    "GRU-MHA (w/o TimesNet)"
]

# 从全局字典中动态获取所有存在的窗口
windows = sorted(list(set([err['window'] for err in all_errors])))

dm_results = {a: {} for a in ablations}

# 3. 遍历计算目标模型与各个消融模型在不同窗口下的 DM 统计量
for w in windows:
    # 获取目标模型在当前窗口的误差序列
    e_target_list = [err['errors'] for err in all_errors if err['model'] == target_model and err['window'] == w]
    if not e_target_list:
        print(f"警告: 未找到目标模型 {target_model} 在 Window={w} 的误差序列。")
        continue
    e_target = e_target_list[-1] 
    
    for b in ablations:
        e_base_list = [err['errors'] for err in all_errors if err['model'] == b and err['window'] == w]
        if not e_base_list:
            dm_results[b][w] = (np.nan, np.nan)
            continue
        e_base = e_base_list[-1]
        
        # 调用基于滚动窗口的 DM 检验
        stat, p = dm_test(e_target, e_base, h=1, loss_type='MSE')
        dm_results[b][w] = (stat, p)

In [95]:
# ---------------- 4. 输出格式模拟 (Rolling-Window DM Test Results) ----------------
print("-" * 135)
header_win = f"{'Ablated/Baseline Model':<40}"
header_met = f"{'':<40}"

for w in windows:
    header_win += f"{f'Window = {w}':^32}"
    header_met += f"{'DM-Stat':>10} {'p-value':>10} {'Sig':>8}  "

print(header_win)
print(header_met)
print("-" * 135)

for b in ablations:
    row = f"{b:<40}"
    for w in windows:
        if w in dm_results[b] and not np.isnan(dm_results[b][w][0]):
            stat, p = dm_results[b][w]
            stars = get_significance_stars(p, stat)
            row += f"{stat:>10.6f} {p:>10.6f} {stars:>8}  "
        else:
            row += f"{'-':>15} {'-':>15} {'':>8}  "
    print(row)
print("-" * 135)
print(f"Note: Target Model = {target_model}.")
print("DM-Stat represents the Diebold-Mariano test statistic with Harvey-Leybourne-Newbold (HLN) correction.")
print("A POSITIVE DM-Stat indicates that the Target Model has a SMALLER loss than the compared model.")
print("Significance levels: *** p<0.01, ** p<0.05, * p<0.1 (Only assigned when the Target Model is strictly better).")

---------------------------------------------------------------------------------------------------------------------------------------
Ablated/Baseline Model                             Window = 5                     Window = 21                     Window = 63           
                                           DM-Stat    p-value      Sig     DM-Stat    p-value      Sig     DM-Stat    p-value      Sig  
---------------------------------------------------------------------------------------------------------------------------------------
TimesNet-GRU-MHA (w/o FFT)                2.465768   0.013851       **    5.422635   0.000000      ***   22.765255   0.000000      ***  
TimesNet-GRU-MHA (w/o 2D Inception)       4.916009   0.000001      ***    2.418999   0.015757       **    4.317074   0.000018      ***  
TimesNet-MHA (w/o GRU)                    2.845392   0.004533      ***   18.476673   0.000000      ***   11.130482   0.000000      ***  
TimesNet-GRU (w/o MHA)                    4

消融实验 (Ablations) 导出

In [ ]:
import pandas as pd
import os

# 提取并扁平化嵌套字典数据
rows = []
for model_name, window_data in dm_results.items():
    for w, (stat, p) in window_data.items():
        stars = get_significance_stars(p, stat) if not np.isnan(stat) else ""
        rows.append({
            "Model": model_name,
            "Window": w,
            "DM_Stat": stat,
            "P_Value": p,
            "Significance": stars
        })

df_dm = pd.DataFrame(rows)

# 直接建目录并保存 CSV
os.makedirs('../results/tables', exist_ok=True)
save_path = '../results/tables/dm_results_ablations.csv'
df_dm.to_csv(save_path, index=False)

print(f"The DM results of Ablations have been saved to: {save_path}")

### w/o Technical Indicators (仅使用原始价格)

### w/o PiT Correction (使用前视偏差数据)

## 保存实验结果

In [96]:
import os
import pandas as pd
import pickle

# ----------------- 1. 导出模型评价指标 (all_results) -----------------
df_results = pd.DataFrame(all_results)
df_results.to_csv('../results/tables/model_evaluation_metrics.csv', index=False)

# ----------------- 2. 导出预测误差序列 (all_errors) -----------------
errors_pkl_path = '../results/tests/model_prediction_errors.pkl'
with open(errors_pkl_path, 'wb') as f:
    pickle.dump(all_errors, f)